# Prediction of pharmacogenomic associations pipeline

This chemoinformatic and bioinformatic pipeline uses datasets to select conserved druggable modules between yeasts and Trypanosomes

----

## Import modules and functions

In [142]:
import pandas as pd
import os
import pipeline_functions
from importlib import reload # reload 
reload(pipeline_functions)

<module 'pipeline_functions' from 'c:\\Users\\Mercedes_IIB-UNSAM\\Desktop\\UNSAM\\yeast_repo_pipeline\\pipeline_functions.py'>

## Import datasets

In [143]:
#Data import
#path = r'c:\\Users\\Mercedes_IIB-UNSAM\\Desktop\\UNSAM\\yeast_repo_pipeline\\data'
path = r'c:\\Users\\Mercedes_IIB-UNSAM\\Desktop\\UNSAM\\yeast_repo_pipeline'
os.chdir(path)
gdi = pd.read_csv("data/gene_drug_interaction",sep='\t')

#Data for genes
yeast_genes_data = pipeline_functions.read_dataframe_columns("data/yeast.genes.list",'\t',['species_code', 'gene_name', 'omcl'])
tryp_genes_data = pipeline_functions.read_dataframe_columns("data/tryp.genes.list",'\t',['species_code', 'gene_name', 'omcl'])
essential_gene_data = pipeline_functions.read_dataframe_columns('data/tbr.essentialOrthologs.list',',')
                                     
#Data for compounds
gdi_inchikey_data = pipeline_functions.smiles_to_inchikey(gdi,'smiles')
gdi_inchikey_data = gdi_inchikey_data[['orf','inchiKey']]                               
tested_compounds_data = pipeline_functions.read_dataframe_columns('data/tested_drugs_inchikey.csv',',')
available_compounds_data = pipeline_functions.read_dataframe_columns('data/drug_availability_inchikey.csv',',')

# Genes filtering

In [144]:
# Assign OMCL to S. cerevisiae genes
genes_data_yeast_intersection = pipeline_functions.data_intersection(gdi_inchikey_data,yeast_genes_data,'orf','gene_name',"OMCL in Yeast",'orf')
genes_data_yeast = genes_data_yeast_intersection[0]
genes_data_yeast = genes_data_yeast[['gene_name','omcl','inchiKey']]
genes_data_yeast = genes_data_yeast.drop_duplicates()
genes_data_yeast = genes_data_yeast.dropna()

In [145]:
# Select T. cruzi genes with shared OMCL
genes_data_yeast_tryp_intersection = pipeline_functions.data_intersection(genes_data_yeast ,tryp_genes_data,'omcl','omcl',"OMCL in Tryps", "omcl")
genes_data_yeast_tryp = genes_data_yeast_tryp_intersection[0]
genes_data_yeast_tryp = genes_data_yeast_tryp[['gene_name_y','omcl','inchiKey']]
genes_data_yeast_tryp = genes_data_yeast_tryp.drop_duplicates()
genes_data_yeast_tryp = genes_data_yeast_tryp.dropna()

In [146]:
# Select Essential genes
genes_data_yeast_tryp_essential_intersection = pipeline_functions.data_intersection(genes_data_yeast_tryp,essential_gene_data,'omcl','omcl',"Essentiality", "omcl")
genes_data_yeast_tryp_essential = genes_data_yeast_tryp_essential_intersection[0]

In [147]:
# Set promiscuity threshold
umbral=200
genes_count = genes_data_yeast_tryp_essential.groupby('gene_name_y').nunique()
genes_count = genes_count.sort_values(by=['inchiKey'])
genes_count = genes_count[genes_count['inchiKey']<umbral]
genes_count = genes_count.unstack(level=-1).reset_index().drop('level_0', axis=1)
genes_count = genes_count.drop_duplicates()
gene_selection = pd.merge(left=gdi_inchikey_data,right=genes_count,how="inner",left_on=['orf'], right_on=['gene_name_y'])
gene_selection = gene_selection[['orf','inchiKey']]

In [148]:
#Create a df with deleted data
not_selected_genes = pd.concat([genes_data_yeast_intersection[1], genes_data_yeast_tryp_intersection[1], genes_data_yeast_tryp_essential_intersection[1]], ignore_index=False)

In [149]:
#See results for selected data
full_dataset_length = gdi_inchikey_data['orf'].nunique()
assign_omcl_length = genes_data_yeast['gene_name'].nunique()
select_t_cruzi_length = genes_data_yeast_tryp['gene_name_y'].nunique()
select_essential_length = genes_data_yeast_tryp_essential['gene_name_y'].nunique()
delete_promiscuous_length = gene_selection['orf'].nunique()

results_selected_genes = pd.DataFrame({
    'Filter': ['Full Dataset Length', 'Assign OMCL to S. cerevisiae genes',
                'Select T. cruzi genes with shared OMCL',
                'Select Essential genes', 'Delete Promiscuous genes'],
    'Count': [full_dataset_length, assign_omcl_length, select_t_cruzi_length,
              select_essential_length, delete_promiscuous_length]
})

print(results_selected_genes)

                                   Filter  Count
0                     Full Dataset Length   5811
1      Assign OMCL to S. cerevisiae genes   5008
2  Select T. cruzi genes with shared OMCL   1780
3                  Select Essential genes   1043
4                Delete Promiscuous genes    996


In [150]:
#See results for not selected data
count_not_selected_genes = not_selected_genes['Filter Name'].value_counts()
results_not_selected_genes = pd.DataFrame(count_not_selected_genes).reset_index()
results_not_selected_genes.columns = ['Filter', 'Count']
print(results_not_selected_genes)

          Filter  Count
0  OMCL in Tryps   2867
1   Essentiality    610
2  OMCL in Yeast    570


# Analysis of genes filter

In [151]:
results_analysis_yeast = pipeline_functions.data_intersection(gdi_inchikey_data,genes_data_yeast,'orf','gene_name','Merged data','orf')

In [152]:
results_analysis_yeast = results_analysis_yeast[0]
results_analysis_yeast = results_analysis_yeast[['gene_name','omcl','inchiKey_y']]
results_analysis_yeast = results_analysis_yeast.drop_duplicates()
results_analysis_yeast = results_analysis_yeast.dropna()

In [153]:
print(len(results_analysis_yeast))
print(len(results_analysis_yeast['gene_name'].unique()))
print(len(results_analysis_yeast['inchiKey_y'].unique()))

234365
5008
2761


In [154]:
results_analysis_yeast_tryp = pipeline_functions.data_intersection(gdi_inchikey_data,genes_data_yeast_tryp,'orf','gene_name_y','Merged data','orf')

In [155]:
results_analysis_yeast_tryp = results_analysis_yeast_tryp[0]
results_analysis_yeast_tryp = results_analysis_yeast_tryp[['gene_name_y','omcl','inchiKey_y']]
results_analysis_yeast_tryp = results_analysis_yeast_tryp.drop_duplicates()
results_analysis_yeast_tryp = results_analysis_yeast_tryp.dropna()

In [156]:
print("All associations with ortholog")
print(len(results_analysis_yeast_tryp))
print("All genes with ortholog")
print(len(results_analysis_yeast_tryp['gene_name_y'].unique()))
print("All compounds with ortholog")
print(len(results_analysis_yeast_tryp['inchiKey_y'].unique()))

All associations with ortholog
87366
All genes with ortholog
1780
All compounds with ortholog
2405


In [157]:
results_analysis_yeast_tryp_essential = pipeline_functions.data_intersection(gdi_inchikey_data,genes_data_yeast_tryp_essential,'orf','gene_name_y','Merged data','orf')

In [158]:
results_analysis_yeast_tryp_essential = results_analysis_yeast_tryp_essential[0]
results_analysis_yeast_tryp_essential = results_analysis_yeast_tryp_essential[['gene_name_y','omcl','inchiKey_y']]
results_analysis_yeast_tryp_essential = results_analysis_yeast_tryp_essential.drop_duplicates()
results_analysis_yeast_tryp_essential = results_analysis_yeast_tryp_essential.dropna()

In [159]:
print("All essential associations")
print(len(results_analysis_yeast_tryp_essential))
print("All essential genes")
print(len(results_analysis_yeast_tryp_essential['gene_name_y'].unique()))
print("All essential compounds")
print(len(results_analysis_yeast_tryp_essential['inchiKey_y'].unique()))

All essential associations
52907
All essential genes
1043
All essential compounds
2171


In [160]:
results_analysis_yeast_tryp_essential_merged = pipeline_functions.data_intersection(gdi_inchikey_data,gene_selection,'orf','orf','Merged data','orf')

In [161]:
results_analysis_yeast_tryp_essential_merged = results_analysis_yeast_tryp_essential_merged[0]
results_analysis_yeast_tryp_essential_merged = results_analysis_yeast_tryp_essential_merged[['orf','inchiKey_y']]
results_analysis_yeast_tryp_essential_merged = results_analysis_yeast_tryp_essential_merged.drop_duplicates()
results_analysis_yeast_tryp_essential_merged = results_analysis_yeast_tryp_essential_merged.dropna()

In [162]:
print("All merged associations")
print(len(results_analysis_yeast_tryp_essential_merged))
print("All merged genes")
print(len(results_analysis_yeast_tryp_essential_merged['orf'].unique()))
print("All merged compounds")
print(len(results_analysis_yeast_tryp_essential_merged['inchiKey_y'].unique()))

All merged associations
39631
All merged genes
996
All merged compounds
1326


# Compounds filtering

In [163]:
# Select novel compounds
compounds_data_tested_intersection = pipeline_functions.data_not_in_intersection(gdi_inchikey_data,tested_compounds_data,'inchiKey','inchikey','Tested compound')
compounds_data_tested = compounds_data_tested_intersection[0]
compounds_data_tested = compounds_data_tested[['orf','inchiKey']]
compounds_data_tested = compounds_data_tested.drop_duplicates()
compounds_data_tested = compounds_data_tested.dropna()

In [164]:
# Select commercially available compounds
compounds_data_tested_available_intersection = pipeline_functions.data_intersection(gdi_inchikey_data,available_compounds_data,'inchiKey','inchikey','Commercially available','inchiKey')
compounds_data_tested_available = compounds_data_tested_available_intersection[0]
compounds_data_tested_available = compounds_data_tested_available[['orf','inchiKey','smiles']]
compounds_data_tested_available = compounds_data_tested_available.drop_duplicates()
compounds_data_tested_available = compounds_data_tested_available.dropna()

In [165]:
# Select drug like and lead like compounds
compounds_data_tested_available_druglike_function = pipeline_functions.drug_likness(compounds_data_tested_available,'smiles')
compounds_data_tested_available_druglike = compounds_data_tested_available_druglike_function[0]
compounds_data_tested_available_druglike = compounds_data_tested_available_druglike[['orf','inchiKey','smiles']]
compounds_data_tested_available_druglike = compounds_data_tested_available_druglike.drop_duplicates()
compounds_data_tested_available_druglike = compounds_data_tested_available_druglike.dropna()

In [166]:
# Set promiscuity threshold
umbral=20
drug_count = compounds_data_tested_available_druglike.groupby('inchiKey').nunique()
drug_count = drug_count.sort_values(by=['orf'])
drug_count = drug_count[drug_count['orf']<umbral]
drug_count = drug_count.unstack(level=-1).reset_index().drop('level_0', axis=1)
drug_count = drug_count.drop_duplicates()

drug_selection = pd.merge(left=gdi_inchikey_data,right=drug_count,how="inner",left_on=['inchiKey'], right_on=['inchiKey'])

In [167]:
#Create a df with deleted data
not_selected_compounds = pd.concat([compounds_data_tested_intersection[1], compounds_data_tested_available_intersection[1], compounds_data_tested_available_druglike_function[1]], ignore_index=False)

In [168]:
drug_selection.head(2)

,orf,inchiKey,0
0,YAR002W,SSPYSWLZOPCOLO-UHFFFAOYSA-N,9
1,YAR002W,SSPYSWLZOPCOLO-UHFFFAOYSA-N,1


In [169]:
#See results for selected data
full_dataset_length = gdi_inchikey_data['inchiKey'].nunique()
tested_length = compounds_data_tested['inchiKey'].nunique()
avaiable_length = compounds_data_tested_available['inchiKey'].nunique()
druglike_length = compounds_data_tested_available_druglike['inchiKey'].nunique()
delete_promiscuous_length = drug_selection['inchiKey'].nunique()

results_selected_compounds = pd.DataFrame({
    'Filter': ['Full Dataset Length', 'Delete tested compounds',
                'Select commercially available compounds',
                'Select druglike compounds', 'Delete Promiscuous compounds'],
    'Count': [full_dataset_length, tested_length, avaiable_length,
              druglike_length, delete_promiscuous_length]
})

print(results_selected_compounds)

                                    Filter  Count
0                      Full Dataset Length   2771
1                  Delete tested compounds   2722
2  Select commercially available compounds   1195
3                Select druglike compounds   1126
4             Delete Promiscuous compounds    745


In [170]:
#See results for not selected data
count_not_selected_compounds = not_selected_compounds['Filter Name'].value_counts()
results_not_selected_compounds = pd.DataFrame(count_not_selected_compounds).reset_index()
results_not_selected_compounds.columns = ['Filter', 'Count']
print(results_not_selected_compounds)

                   Filter  Count
0            Not Druglike  30014
1         Tested compound  24570
2  Commercially available   1576


# Analysis of compounds filter

In [ ]:
results_analysis_tested = pipeline_functions.data_intersection(gdi_inchikey_data,compounds_data_tested,'inchiKey','inchiKey','Merged data','inchiKey')

In [ ]:
results_analysis_tested = results_analysis_tested[0]
results_analysis_tested = results_analysis_tested[['orf_y','inchiKey']]
results_analysis_tested = results_analysis_tested.drop_duplicates()
results_analysis_tested = results_analysis_tested.dropna()

In [ ]:
print("All novel associations")
print(len(results_analysis_tested))
print("All novel genes")
print(len(results_analysis_tested['orf_y'].unique()))
print("All novel compounds")
print(len(results_analysis_tested['inchiKey'].unique()))

5220
1195


In [172]:
results_analysis_tested_available = pipeline_functions.data_intersection(gdi_inchikey_data,compounds_data_tested_available,'inchiKey','inchiKey','Merged data','inchiKey')

In [ ]:
results_analysis_tested_available = results_analysis_tested_available[0]
results_analysis_tested_available = results_analysis_tested_available[['orf_y','inchiKey']]
results_analysis_tested_available = results_analysis_tested_available.drop_duplicates()
results_analysis_tested_available = results_analysis_tested_available.dropna()

In [ ]:
print("All available associations")
print(len(results_analysis_tested_available))
print("All available genes")
print(len(results_analysis_tested_available['orf_y'].unique()))
print("All available compounds")
print(len(results_analysis_tested_available['inchiKey'].unique()))

5115
1126


In [ ]:
compounds_data_tested_available_druglike = pipeline_functions.data_intersection(gdi_inchikey_data,compounds_data_tested_available_druglike,'inchiKey','inchiKey','Merged data','inchiKey')

In [ ]:
compounds_data_tested_available_druglike = compounds_data_tested_available_druglike[0]
compounds_data_tested_available_druglike = compounds_data_tested_available_druglike[['orf_y','inchiKey']]
compounds_data_tested_available_druglike = compounds_data_tested_available_druglike.drop_duplicates()
compounds_data_tested_available_druglike = compounds_data_tested_available_druglike.dropna()

In [ ]:
print("All druglike associations")
print(len(compounds_data_tested_available_druglike))
print("All druglike genes")
print(len(compounds_data_tested_available_druglike['orf_y'].unique()))
print("All druglike compounds")
print(len(compounds_data_tested_available_druglike['inchiKey'].unique()))

310
745


# Data merge

In [174]:
results = pipeline_functions.data_intersection(drug_selection,gene_selection,'inchiKey','inchiKey','Merged data','inchiKey')

In [175]:
results_selected = results[0]
results_selected = results_selected[['orf_x','inchiKey']]
results_selected = results_selected.rename(columns={'orf_x':'orf'}) 
results_selected = results_selected.drop_duplicates()
results_selected = results_selected.dropna()

In [178]:
print("All merged associations")
print(len(results_selected))
print("All merged genes")
print(len(results_selected['orf'].unique()))
print("All merged compounds")
print(len(results_selected['inchiKey'].unique()))

All merged associations
246
All merged genes
73
All merged compounds
189


In [180]:
results_selected.to_csv('results/gdi_results.csv')

In [181]:
not_selected_data = pd.concat([results[1], not_selected_genes, not_selected_compounds], ignore_index=False)

In [182]:
not_selected_data = not_selected_data[['inchiKey','orf','Filter Name']]

In [183]:
#See results for not selected data
count_not_selected_data = not_selected_data['Filter Name'].value_counts()
results_not_selected_data = pd.DataFrame(count_not_selected_data).reset_index()
results_not_selected_data.columns = ['Filter', 'Count']
print(results_not_selected_data)

                   Filter  Count
0            Not Druglike  30014
1         Tested compound  24570
2           OMCL in Tryps   2867
3  Commercially available   1576
4            Essentiality    610
5           OMCL in Yeast    570
6             Merged data    556


In [184]:
not_selected_data.to_csv('results/gdi_not_selected_data.csv')